# **spaCy object extraction technique**

In [ ]:
!pip install spacy pandas
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 107.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import spacy
import pandas as pd
import tqdm
import torch
from google.colab import files

if torch.cuda.is_available():
    spacy.require_gpu()
    nlp = spacy.load("en_core_web_trf")
    print("Using GPU with Transformer-based model for SpaCy processing.")
else:
    nlp = spacy.load("en_core_web_sm")
    print("Using CPU for SpaCy processing.")

csv_path = "/content/drive/MyDrive/Grad/CAP6412-0001/LLaVA_v1.5_Values.csv"
df = pd.read_csv(csv_path, header=None, names=["text"])

# Function to extract objects (nouns)
def extract_objects(text):
    doc = nlp(text)
    return [token.text for token in doc if token.pos_ == "NOUN"]

batch_size = 5000
objects_list = []
# max_objects = 5  # Limit for testing

for batch_start in tqdm.tqdm(range(0, len(df), batch_size), desc="Extracting Objects in Batches"):
    batch_texts = df["text"][batch_start:batch_start + batch_size].astype(str).tolist()
    docs = list(nlp.pipe(batch_texts, batch_size=batch_size))

    for doc in docs:
        objects = [token.text for token in doc if token.pos_ == "NOUN"]
        objects_list.extend(objects)

    #     if len(objects_list) >= max_objects:
    #         break
    # if len(objects_list) >= max_objects:
    #     break

# Creating frequency count
object_counts = pd.Series(objects_list).value_counts().reset_index()
object_counts.columns = ["Object", "Frequency"]

output_csv_path = "Extracted_Objects.csv"
object_counts.to_csv(output_csv_path, index=False)

# results
print("\nTop 10 extracted objects:")
print(object_counts.head(10))

print("\nDownloading extracted objects CSV file...")
files.download(output_csv_path)


Using GPU with Transformer-based model for SpaCy processing.


Extracting Objects in Batches:  98%|█████████▊| 1344/1378 [4:27:53<06:46, 11.96s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 366.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 164.88 MiB is free. Process 140710 has 39.39 GiB memory in use. Of the allocated memory 36.76 GiB is allocated by PyTorch, and 2.13 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)